In [32]:
import pandas as pd
import numpy as np
import json
import requests
%matplotlib inline

# models by tag type

In [55]:
from sklearn.model_selection import train_test_split
#corpus train/test
data = pd.read_csv('handicap_2019_06_19_keywordsstakes2.csv', encoding='utf-8')
props=list(data['content'])
keywords=list(data['keywords'])
stakes=list(data['stake_extrait'])
data = data[pd.notnull(data['stake_extrait'])]
X_train, X_test, y_train, y_test = train_test_split(data[["content"]],data['stake_extrait'],test_size=0.3)

In [56]:
print(list(X_train['content'])[:10])
print(list(y_train)[:10])

["Il faut que les AVS et les instituteurs suivent de meilleures formations aux différents DYS, et à comment adapter à l'école ", "Il faut accompagner la création d'association d'aide aux aidants et personnes concernées par un handicap, des financement par activité", 'Il faut arrêter de priver d’AAH ou autres prestations aux ressources du conjoint. ', 'Il faut créer des écoles en langue des signes française ', 'Il faut que tous les sites publics et parapublics soient accessibles... disons en 2021 ? Chiche !', "Il faut que les transports en commun respectent l'information vocale aux passagers", 'Il faut banaliser la scolarité des enfants en situation de handicap et non en faire une exception ou une anormalité.', 'Il faut faire changer le regard des professionnels qui pensent "handicapé = aménagements coûteux, problème d\'intégration et de compétences"', 'Il faut que les handicapés puissent vivre comme les valides en n’étant pas handicapés par l’inadaptation des lieux publics', 'Il faut q

In [57]:
X_train=list(X_train['content'])
y_train=list(y_train)
X_test=list(X_test['content'])

In [58]:
import stop_words
stop_words_fr = stop_words.get_stop_words('fr')

In [59]:
stop_words_fr += ['il', 'faut', 'plus']

In [60]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words=stop_words_fr)
#max_df=.2, min_df=2
X_train_f = tfidf.fit_transform(X_train)

In [61]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB(fit_prior=False)
nb.fit(X_train_f, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=False)

In [62]:
X_pred = tfidf.transform(X_test)

In [63]:
y_pred = nb.predict(X_pred)

In [64]:
y_proba = nb.predict_proba(X_pred)

In [72]:
classe2id={t:i for i,t in enumerate(nb.classes_) }
print(nb.classes_)

['accessibilité et mobilité' 'aidants' 'allocations et financier'
 'changement regard' 'handicaps invisibles' 'prise en charge' 'travail'
 'éducation']


In [77]:
res=0
for i in range(len(X_test)) :
    print(X_test[i], ":", list(y_test)[i], "=>", y_pred[i], y_proba[i][classe2id[y_pred[i]]], "\n")
    if(y_pred[i]==list(y_test)[i]) :
        res+=1

Il faut que l’AAH devienne un revenu existentiel  : allocations et financier => allocations et financier 0.46394519731203016 

Il faut faciliter l’accès à l’emploi  : travail => travail 0.2770240766124012 

Il faut que tous les documents, sites internet, applications, cartes et signalisations soient accessibles à tous, y compris aux daltoniens : accessibilité et mobilité => accessibilité et mobilité 0.26606568023644367 

Il faut des aides aux devoirs encadrées et accessibles financièrement par des pers. formées aux handicap (épilepsie, trouble de l’attention) : accessibilité et mobilité => accessibilité et mobilité 0.15780125566617523 

Il faut former les personnels des MDPH à l'évaluation des besoins : allocations et financier => éducation 0.2332843243086925 

Il faut ne plus prendre en compte les ressources du conjoint pour le bénéficiaire de l'AAH. C'est le rendre dépendant encore plus : allocations et financier => allocations et financier 0.7052064254041286 

Il faut que les conjoi

Il faut une prise en charge plus rapide, que les enfants soient suivis par une personne tout le long de la journée (garderie, ALSH, école..) : prise en charge => prise en charge 0.2929927365016812 

Il faut aider les aidants à avoir une petite activité professionnelle compatible avec leur rôle d'aidant : aidants => aidants 0.28688650066744403 

Il faut construire les PRITH plan régionaux d'insertion des travailleurs handicapés avec les associations  : travail => travail 0.30062639426947496 

Il faut inclure dans le programme de formation des enseignants du primaire à l'université un module de sensibilisation  au handicap  : éducation => éducation 0.47281469291150463 

Il faut prévoir une personne aidante auprès des professeurs afin de faciliter la disponibilité envers ces enfants : éducation => éducation 0.32482182354083655 

Il faut communiquer sur les compétences des travailleurs handicapés d’esat et EA à destination des entreprises pour encourager l’inclusion : travail => travail 0.

In [81]:
print(res/len(X_test)*100,'%')

88.8 %


In [82]:
res=0
for i in range(len(X_test)) :
    if(y_pred[i]!=list(y_test)[i]) :
        print(X_test[i], ":", list(y_test)[i], "=>", y_pred[i], y_proba[i][classe2id[y_pred[i]]], "\n")

Il faut former les personnels des MDPH à l'évaluation des besoins : allocations et financier => éducation 0.2332843243086925 

Il faut que les enseignants spécialisés enseignent et arrêtent de remplir des GEVA Sco à la place de la MDPH. Comme en Italie : allocations et financier => éducation 0.21477895603991626 

Il faut permettre au maximum, que les parents d'enfants handicapés conservent leur travail : travail => éducation 0.26752132382089155 

Il faut intégrer dans les "AIDES HUMAINES" la prise en compte d'une aide administrative pour une personne ayant un handicap invisible : handicaps invisibles => allocations et financier 0.31683094069163126 

Il faut former les profs de façon obligatoire aux troubles Dys.  : handicaps invisibles => éducation 0.35013554102284683 

Il faut créer plus de cabinets médicaux (dentistes, podologues, kinésithérapeutes, dermatologues, radiologues) accessibles aux handicapés : accessibilité et mobilité => éducation 0.2649465409082837 

Il faut reconnaître